In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.tools import tool, ToolRuntime

@tool
def read_email(runtime: ToolRuntime) -> str:
    """Read the current email from the inbox. Returns the full email content."""
    # take email from state
    return runtime.state["email"]

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send an email reply.
    
    Args:
        to: The recipient's name
        subject: The email subject line
        body: The complete email body including greeting and signature
    """
    # fake email sending
    return f"Email sent successfully to {to}"

In [29]:
from langchain.agents import create_agent, AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="llama3.1:8b",
    temperature=0.0,
)


class EmailState(AgentState):
    email: str


email_agent = create_agent(
    model=model,
    tools=[read_email, send_email],
    state_schema=EmailState,
    checkpointer=InMemorySaver(),
    system_prompt="""You are pytholic's email assistant. 

CRITICAL: You MUST use the send_email tool to send responses. Never write email text directly - always call the send_email tool.

Your workflow:
1. Use read_email tool to read the incoming email
2. Analyze the email content and sender's name
3. Use send_email tool with these parameters:
   - to: recipient's name (from the email)
   - subject: appropriate subject line (e.g., "Re: [original topic]")
   - body: complete professional email with greeting, full response, and signature

Email writing guidelines:
- Address all questions or requests
- Be polite and professional
- Sign as "pytholic"
- Include proper greeting and closing

Remember: ALWAYS use the send_email tool. Do not write email content as plain text.""",
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "read_email": False,
                "send_email": True,
            },
            description_prefix="Tool execution requires approval",
        ),
    ],
)


In [30]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = email_agent.invoke(
    {
        "messages": [HumanMessage(content="Read my email and send a response using the send_email tool.")],
        "email": "Hi pytholic, I'm going to be late for our meeting tomorrow. Can we reschedule? Best, John."
    },
    config=config
)

In [31]:
from pprint import pprint

pprint(response)

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': 'Dear '
                                                                          'John,\n'
                                                                          '\n'
                                                                          'Thank '
                                                                          'you '
                                                                          'for '
                                                                          'reaching '
                                                                          'out '
                                                                          'about '
                                                                          'the '
                                                                          'potential '
                                                                          'delay '
         

In [24]:
print(response['__interrupt__'])

[Interrupt(value={'action_requests': [{'name': 'send_email', 'args': {'to': 'John', 'subject': "Re: Rescheduling Tomorrow's Meeting", 'body': "Dear John,\n\nThank you for reaching out about the potential delay in our meeting tomorrow. I understand that unforeseen circumstances can arise, and I'm more than happy to reschedule at your convenience.\n\nPlease let me know a few alternative dates and times that work for you, and I'll do my best to accommodate them.\n\nLooking forward to hearing back from you soon.\n\nBest regards,\npytholic"}, 'description': 'Tool execution requires approval\n\nTool: send_email\nArgs: {\'to\': \'John\', \'subject\': "Re: Rescheduling Tomorrow\'s Meeting", \'body\': "Dear John,\\n\\nThank you for reaching out about the potential delay in our meeting tomorrow. I understand that unforeseen circumstances can arise, and I\'m more than happy to reschedule at your convenience.\\n\\nPlease let me know a few alternative dates and times that work for you, and I\'ll do

In [7]:
# Access just the 'body' argument from the tool call
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

Dear John,

Thank you for reaching out about the potential delay in our meeting tomorrow. I understand that unforeseen circumstances can arise, and I'm more than happy to reschedule at your convenience.

Please let me know a few alternative dates and times that work for you, and I'll do my best to accommodate them.

Looking forward to hearing back from you soon.

Best regards,
pytholic


## Approve

In [8]:
from langgraph.types import Command

command = Command(resume={"decisions": [{"type": "approve"}]})

response = email_agent.invoke(
    input=command,
    config=config # Same thread_id to resume the same conversation
)

pprint(response)

{'email': "Hi pytholic, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Read my email and send a response using the send_email tool.', additional_kwargs={}, response_metadata={}, id='5064815e-f69b-4d8d-9825-0cd8500804da'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2026-01-27T03:06:32.695368Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3321624833, 'load_duration': 151597333, 'prompt_eval_count': 403, 'prompt_eval_duration': 1866833167, 'eval_count': 35, 'eval_duration': 1285064790, 'logprobs': None, 'model_name': 'llama3.1:8b', 'model_provider': 'ollama'}, id='lc_run--019bfd6a-bc77-7950-9743-6ef5beac9056-0', tool_calls=[{'name': 'read_email', 'args': {}, 'id': 'c2b0185c-f4ef-4e2f-a083-cf050cc361d1', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 403, 'output_tokens': 35, 'total_tokens': 438}

In [9]:
print(response['messages'][-1].content)

Would you like me to read another email?


## Reject

In [14]:
response = email_agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "reject",
                    # An explanation of why the request was rejected
                    "message": "No please sign off - Your merciful leader, pytholic."
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': 'Dear '
                                                                          'John,\n'
                                                                          '\n'
                                                                          'Thank '
                                                                          'you '
                                                                          'for '
                                                                          'reaching '
                                                                          'out '
                                                                          'about '
                                                                          'the '
                                                                          'potential '
                                                                          'delay '
         

In [15]:
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

Dear John,

Thank you for reaching out about the potential delay in our meeting tomorrow. I understand that unforeseen circumstances can arise, and I'm more than happy to reschedule at your convenience.

Please let me know a few alternative dates and times that work for you, and I'll do my best to accommodate them.

Looking forward to hearing back from you soon.

Your Merciful Leader,
pytholic


## Edit

In [16]:
response = email_agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "edit",
                    # Edited action with tool name and args
                    "edited_action": {
                        # Tool name to call.
                        # Will usually be the same as the original action.
                        "name": "send_email",
                        # Arguments to pass to the tool.
                        "args": {
                            "subject": "Re: Meeting Rescheduling",
                            "body": "This is the last straw, you're fired!"
                            },
                    }
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': 'Dear '
                                                                          'John,\n'
                                                                          '\n'
                                                                          "I'm "
                                                                          'afraid '
                                                                          'I '
                                                                          'must '
                                                                          'correct '
                                                                          'my '
                                                                          'previous '
                                                                          'response. '
                                                                          'As '
           

The tool call failed due to missing args.

## Edit - Partial Args Helper

**Why do I need all args for edit?**

The three decision types work differently:
- **`approve`**: No args needed - just approves the model's proposed action
- **`reject`**: Only needs a `message` - rejects and provides feedback to the model
- **`edit`**: Requires **ALL** tool parameters - completely replaces the tool call

When using `"type": "edit"`, you must provide **all** required tool arguments because you're replacing the entire tool call. 

**Solution: Extract and merge** - To make partial edits easier, extract the original args first and only override what you want to change:

In [32]:
response['__interrupt__'][0].value['action_requests'][0]

{'name': 'send_email',
 'args': {'to': 'John',
  'subject': "Re: Rescheduling Tomorrow's Meeting",
  'body': "Dear John,\n\nThank you for reaching out about the potential delay in our meeting tomorrow. I understand that unforeseen circumstances can arise, and I'm more than happy to reschedule at your convenience.\n\nPlease let me know a few alternative dates and times that work for you, and I'll do my best to accommodate them.\n\nLooking forward to hearing back from you soon.\n\nBest regards,\npytholic"},
 'description': 'Tool execution requires approval\n\nTool: send_email\nArgs: {\'to\': \'John\', \'subject\': "Re: Rescheduling Tomorrow\'s Meeting", \'body\': "Dear John,\\n\\nThank you for reaching out about the potential delay in our meeting tomorrow. I understand that unforeseen circumstances can arise, and I\'m more than happy to reschedule at your convenience.\\n\\nPlease let me know a few alternative dates and times that work for you, and I\'ll do my best to accommodate them.\\n

In [33]:
# Extract original args from the interrupt
original_args = response['__interrupt__'][0].value['action_requests'][0]['args']
print("Original args:", original_args)

# Create new args by merging - only override what you want to change
edited_args = {
    **original_args,  # Keep all original args
    "body": "The meeting cannot be rescheduled. If you do not show, there will be consequences"  # Override just the body
}

print("\nEdited args:", edited_args)

Original args: {'to': 'John', 'subject': "Re: Rescheduling Tomorrow's Meeting", 'body': "Dear John,\n\nThank you for reaching out about the potential delay in our meeting tomorrow. I understand that unforeseen circumstances can arise, and I'm more than happy to reschedule at your convenience.\n\nPlease let me know a few alternative dates and times that work for you, and I'll do my best to accommodate them.\n\nLooking forward to hearing back from you soon.\n\nBest regards,\npytholic"}

Edited args: {'to': 'John', 'subject': "Re: Rescheduling Tomorrow's Meeting", 'body': 'The meeting cannot be rescheduled. If you do not show, there will be consequences'}


In [34]:
# Now use the edited_args in your edit decision
response = email_agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "edit",
                    "edited_action": {
                        "name": "send_email",
                        "args": edited_args  # Use the merged args
                    }
                }
            ]
        }
    ), 
    config=config
)

pprint(response)

{'email': "Hi pytholic, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Read my email and send a response using the send_email tool.', additional_kwargs={}, response_metadata={}, id='82f28b08-b1db-48bc-a30e-5fd72d363e1d'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2026-01-27T03:10:56.049317Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3099307875, 'load_duration': 140990958, 'prompt_eval_count': 403, 'prompt_eval_duration': 1643290708, 'eval_count': 35, 'eval_duration': 1295731041, 'logprobs': None, 'model_name': 'llama3.1:8b', 'model_provider': 'ollama'}, id='lc_run--019bfd6e-c20d-7063-b661-a713bcaf2c1a-0', tool_calls=[{'name': 'read_email', 'args': {}, 'id': 'a515b886-f4d7-4a89-a1a6-51e7258b675c', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 403, 'output_tokens': 35, 'total_tokens': 438}

Trace: https://smith.langchain.com/public/c50fc684-7a73-4f10-96cb-7ac2fd4ba7a9/r

Note: 
- I recommend re-running the initial email call before each part (approve ,reject, edit) to have clear context.